In [4]:

!pip install keras


In [6]:
#Definition of library
!pip install pykdtree
!pip install kdtree

import os
import cv2
import csv
import argparse
from scipy.io import loadmat
import csv
import math
import random
from scipy.io import loadmat
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.models import load_model
import pandas as pd
import numpy as np
import keras.backend as K
from pykdtree.kdtree import KDTree
import glob
import h5py
from scipy.io import loadmat
from tqdm import tqdm
import h5py
import scipy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
%matplotlib inline
#from utils_gen import gen_density_map_gaussian

In [7]:
def gen_density_map(img, anno_points):
    # Görüntü yoksa veya boşsa hata döndür
    if img is None or img.size == 0:
        raise ValueError("Image is None or empty.")

    # Görüntü boyutunu al
    h, w = img.shape[:2]

    # Yoğunluk haritasını oluştur
    density_map = np.zeros((h, w), dtype=np.float64)
    kernel_size = 15  # Gaussian kernel boyutu
    sigma = 4.0  # standart sapma

    for point in anno_points:
        # İnsan başının merkez noktası koordinatları
        x, y = min(w - 1, abs(math.floor(point[0]))), min(h - 1, abs(math.floor(point[1])))
        # Üst sol köşe koordinatları ve alt sağ köşe koordinatları
        x1, y1 = max(0, x - kernel_size // 2), max(0, y - kernel_size // 2)
        x2, y2 = min(w, x + kernel_size // 2 + 1), min(h, y + kernel_size // 2 + 1)

        # Sınırlar dışına çıkılıp çıkılmadığını kontrol et
        out_of_bounds = False
        dx1, dy1, dx2, dy2 = 0, 0, 0, 0  # Sınırlar dışı ofset
        if x1 < 0:
            dx1 = abs(x1)
            x1 = 0
            out_of_bounds = True
        if y1 < 0:
            dy1 = abs(y1)
            y1 = 0
            out_of_bounds = True
        if x2 > w:
            dx2 = x2 - w
            x2 = w
            out_of_bounds = True
        if y2 > h:
            dy2 = y2 - h
            y2 = h
            out_of_bounds = True

        if out_of_bounds:
            # Sınırlar dışına çıkıldığında, Gauss çekirdeğinin boyutunu ayarla
            kernel_h = kernel_size - dy1 - dy2
            kernel_w = kernel_size - dx1 - dx2
            # (kernel_h, kernel_w) boyutunda bir Gauss çekirdeği oluştur
            H = np.multiply(cv2.getGaussianKernel(kernel_h, sigma), (cv2.getGaussianKernel(kernel_w, sigma)).T)
        else:
            # (15, 15) boyutunda bir Gauss çekirdeği oluştur
            H = np.multiply(cv2.getGaussianKernel(kernel_size, sigma), (cv2.getGaussianKernel(kernel_size, sigma)).T)

        density_map[y1:y2, x1:x2] += H

    return density_map



class DataLoader(object):
    def __init__(self, data_path, gt_path, shuffle=False, gt_downsample=False):
     
        self.data_path = data_path
        self.gt_path = gt_path
        self.shuffle = shuffle
        self.gt_downsample = gt_downsample
        self.data_files = [filename for filename in os.listdir(data_path)]
        self.num_samples = len(self.data_files)
        self.blob_list = []

        for fname in self.data_files:
            img = cv2.imread(os.path.join(self.data_path, fname), 0)
            img = img.astype(np.float32, copy=False)
            ht = img.shape[0]
            wd = img.shape[1]
            ht_1 = int((ht / 4) * 4)
            wd_1 = int((wd / 4) * 4)
            img = cv2.resize(img, (wd_1, ht_1))
            img = img.reshape((img.shape[0], img.shape[1], 1))
            den = pd.read_csv(os.path.join(self.gt_path, os.path.splitext(fname)[0] +'.csv'),
                              header=None).values
            den = den.astype(np.float32, copy=False)
            if self.gt_downsample:
                wd_1 = int(wd_1 / 4)
                ht_1 = int(ht_1 / 4)
            den = cv2.resize(den, (wd_1, ht_1))
            den = den * ((wd * ht) / (wd_1 * ht_1))
            den = den.reshape((den.shape[0], den.shape[1], 1))

            blob = dict()
            blob['data'] = img
            blob['gt'] = den
            blob['fname'] = fname
            self.blob_list.append(blob)

        if self.shuffle:
            np.random.shuffle(self.blob_list)

    def flow(self, batch_size=32):
        loop_count = self.num_samples // batch_size
        while True:
            np.random.shuffle(self.blob_list)
            for i in range(loop_count):
                blobs = self.blob_list[i*batch_size: (i+1)*batch_size]
                X_batch = np.array([blob['data'] for blob in blobs])
                Y_batch = np.array([blob['gt'] for blob in blobs])
                yield X_batch, Y_batch

    def get_all(self):
        X = np.array([blob['data'] for blob in self.blob_list])
        Y = np.array([blob['gt'] for blob in self.blob_list])
        return X, Y

    def __iter__(self):
        for blob in self.blob_list:
            yield blob

def mae(y_true, y_pred):
    return K.abs(K.sum(y_true) - K.sum(y_pred))

def mse(y_true, y_pred):
    return (K.sum(y_true) - K.sum(y_pred)) * (K.sum(y_true) - K.sum(y_pred))

In [10]:
import os

EPOCHS = 200
TRAIN_BATCH_SIZE = 1
VAL_BATCH_SIZE = 1
dataset = 'A'
MODEL_DIR = './trained_models/'

TRAIN_PATH = r"C:\Users\bakiD\OneDrive\Masaüstü\ShanghaiTech\part_A\train_data\images"
TRAIN_GT_PATH = r"C:\Users\bakiD\OneDrive\Masaüstü\ShanghaiTech\part_A\train_data\ground-truth"

VAL_PATH = r"C:\Users\bakiD\OneDrive\Masaüstü\ShanghaiTech\part_A\val_data\images"
VAL_GT_PATH = r"C:\Users\bakiD\OneDrive\Masaüstü\ShanghaiTech\part_A\val_data\ground-truth"

TEST_PATH = r"C:\Users\bakiD\OneDrive\Masaüstü\ShanghaiTech\part_A\test_data\images"
TEST_GT_PATH = r"C:\Users\bakiD\OneDrive\Masaüstü\Shangtai\part_A\test_data\ground-truth"

HM_GT_PATH = r"C:\Users\bakiD\OneDrive\Masaüstü\Shangtai\part_A\Heat_map"

path = r"C:\Users\bakiD\OneDrive\Masaüstü\Shangtai\part_A\train_data\images"
gt_path = r"C:\Users\bakiD\OneDrive\Masaüstü\Shangtai\part_A\train_data\ground-truth"
gt_path_csv = r"C:\Users\bakiD\OneDrive\Masaüstü\Shangtai\part_A/ground_truth_csv/"

if not os.path.exists(gt_path_csv):
    os.makedirs(gt_path_csv)
#num image
num_images = 300
x = ''.join((gt_path, 'GT_IMG_1', '.mat'))
print(x)


C:\Users\bakiD\OneDrive\Masaüstü\Shangtai\part_A\train_data\ground-truthGT_IMG_1.mat


In [11]:
num_val = math.ceil(num_images * 0.1)  
indices = list(range(1, num_images + 1))
random.shuffle(indices)

for idx in range(num_images):


    i = indices[idx]
    if (idx+1) % 10 == 0:
        print('Processing {}/{} files'.format(idx+1, num_images))
    
    input_img_name = ''.join((path, 'IMG_', str(i), '.jpg'))
    if os.path.isfile(input_img_name):

      im = cv2.imread(input_img_name, 0)
      
      image_info = loadmat(''.join((gt_path, 'GT_IMG_', str(i), '.mat')))['image_info']
      annPoints = image_info[0][0][0][0][0] - 1
      
      im_density = gen_density_map.gen_density_map(im, annPoints)

      h, w = im.shape
      wn2, hn2 = w / 8, h / 8  
      wn2, hn2 = int(wn2 / 8) * 8, int(hn2 / 8) * 8  
      
      xmin, xmax = wn2, w - wn2
      ymin, ymax = hn2, h - hn2
      
      for j in range(1, N + 1):
          
          x = math.floor((xmax - xmin) * random.random() + xmin)
          y = math.floor((ymax - ymin) * random.random() + ymin)
          
          x1, y1 = x - wn2, y - hn2
          x2, y2 = x + wn2, y + hn2
          
          im_sampled = im[y1:y2, x1:x2]
          im_density_sampled = im_density[y1:y2, x1:x2]

          
          img_idx = ''.join((str(i), '_', str(j)))
          path_img, path_den = (val_path_img, val_path_den) if (idx+1) < num_val else (train_path_img, train_path_den)
          cv2.imwrite(''.join([path_img, img_idx, '.jpg']), im_sampled)
          with open(''.join([path_den, img_idx, '.csv']), 'w', newline='') as fout:
              writer = csv.writer(fout)
              writer.writerows(im_density_sampled)

Processing 10/300 files
Processing 20/300 files
Processing 30/300 files
Processing 40/300 files
Processing 50/300 files
Processing 60/300 files
Processing 70/300 files
Processing 80/300 files
Processing 90/300 files
Processing 100/300 files
Processing 110/300 files
Processing 120/300 files
Processing 130/300 files
Processing 140/300 files
Processing 150/300 files
Processing 160/300 files
Processing 170/300 files
Processing 180/300 files
Processing 190/300 files
Processing 200/300 files
Processing 210/300 files
Processing 220/300 files
Processing 230/300 files
Processing 240/300 files
Processing 250/300 files
Processing 260/300 files
Processing 270/300 files
Processing 280/300 files
Processing 290/300 files
Processing 300/300 files


In [12]:
dataset = 'A'
N = 9  
dataset_name = ''.join(['ShanghaiTech_part_', dataset, '_patches_', str(N)])
path = ''.join(['ShanghaiTech/part_', dataset, '/train_data/images/'])
output_path = 'data/formatted_trainval_{}/'.format(dataset)
train_path_img = ''.join((output_path, dataset_name, '/train/'))
train_path_den = ''.join((output_path, dataset_name, '/train_den/'))
val_path_img = ''.join((output_path, dataset_name, '/val/'))
val_path_den = ''.join((output_path, dataset_name, '/val_den/'))
gt_path = ''.join(['ShanghaiTech/part_', dataset, '/train_data/ground-truth/'])
print(gt_path)

for i in [output_path, train_path_img, train_path_den, val_path_img, val_path_den]:
    if not os.path.exists(i):
        os.makedirs(i)

ShanghaiTech/part_A/train_data/ground-truth/


In [16]:
import numpy as np

seed = 26
random.seed(seed)
dataset = 'A'
N = 9  
dataset_name = ''.join(['ShanghaiTech_part_', dataset, '_patches_', str(N)])
path = r"C:/Users/bakiD/OneDrive/Masaüstü/ShanghaiTech/part_A/train_data/images/"
output_path = 'data/formatted_trainval_{}/'.format(dataset)
train_path_img = ''.join((output_path, dataset_name, '/train/'))
train_path_den = ''.join((output_path, dataset_name, '/train_den/'))
val_path_img = ''.join((output_path, dataset_name, '/val/'))
val_path_den = ''.join((output_path, dataset_name, '/val_den/'))
gt_path = r"C:/Users/bakiD/OneDrive/Masaüstü/ShanghaiTech/part_A/train_data/ground-truth/"
print(gt_path)
for i in [output_path, train_path_img, train_path_den, val_path_img, val_path_den]:
    if not os.path.exists(i):
        os.makedirs(i)

if dataset == 'A':
    num_images = 300
else:
    num_images = 400

num_val = math.ceil(num_images * 0.1) 
indices = list(range(1, num_images + 1))
random.shuffle(indices)

for idx in range(num_images):

  i = indices[idx]
  if (idx+1) % 10 == 0:
      print('Processing {}/{} files'.format(idx+1, num_images))
  
  input_img_name = ''.join((path, 'IMG_', str(i), '.jpg'))
  im = cv2.imread(input_img_name, 0)  


  
  x11=''.join((gt_path, 'GT_IMG_', str(i), '.mat'))

  image_info = loadmat(''.join((gt_path, 'GT_IMG_', str(i), '.mat')))['image_info']
  annPoints = image_info[0][0][0][0][0] - 1

  density_map = np.zeros_like(im, dtype=np.float64)
  print(density_map.shape)
  
  im_density = gen_density_map(im, annPoints)
  
  h, w = im.shape
  wn2, hn2 = w / 8, h / 8  
  wn2, hn2 = int(wn2 / 8) * 8, int(hn2 / 8) * 8 
  xmin, xmax = wn2, w - wn2
  ymin, ymax = hn2, h - hn2
  for j in range(1, N + 1):
      x = math.floor((xmax - xmin) * random.random() + xmin)
      y = math.floor((ymax - ymin) * random.random() + ymin)
      x1, y1 = x - wn2, y - hn2
      x2, y2 = x + wn2, y + hn2
     
      im_sampled = im[y1:y2, x1:x2]
      im_density_sampled = im_density[y1:y2, x1:x2]
     
      img_idx = ''.join((str(i), '_', str(j)))
      path_img, path_den = (val_path_img, val_path_den) if (idx+1) < num_val else (train_path_img, train_path_den)
      cv2.imwrite(''.join([path_img, img_idx, '.jpg']), im_sampled)
      with open(''.join([path_den, img_idx, '.csv']), 'w', newline='') as fout:
          writer = csv.writer(fout)
          writer.writerows(im_density_sampled)

C:/Users/bakiD/OneDrive/Masaüstü/ShanghaiTech/part_A/train_data/ground-truth/
()


ValueError: Image is None or empty.